In [ ]:
import concurrent.futures
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from functools import partial

sys.path.append('..')

from change_of_basis import cmat_of_tmat
from change_of_basis import tmat_of_cmat
from ES_ContourPlots import ES_ContourPlots
from find_symmetry_groups import UsHat
from find_symmetry_groups import proj_to_vsig_of_u_new
from materials import get_materials_Cvec
from safe_module import closest
from safe_module import distance
from safe_module import GetTempAndT0S0P0
from themes import colors
from themes import symmetry_classes
from utilities import sm2v
from utilities import v2sm

In [ ]:
plt.rcParams['lines.markersize'] = 3

In [ ]:
write = False

In [ ]:
pathway = 3

if pathway==1:
    
    node_sequence    = ['TRIV','ISO']       # Note that this should always be a list of length 2
    node_mode        = 1
    t_min            = -3/5
    t_max            = 1
    dt               = 1/5

elif pathway==2:
    
    node_sequence    = ['TRIV','MONO','ORTH','TET','XISO','ISO']
    node_mode        = 1
    t_min            = 0
    t_max            = 1
    dt               = 1/5

elif pathway==3:
    
    node_sequence    = ['TRIV','MONO','ORTH','TET','XISO','ISO']
    node_mode        = 2
    t_min            = 0
    t_max            = 1
    dt               = 1/5
    
elif pathway==4:
    
    node_sequence    = ['TRIV','MONO','ORTH','TET','XISO','ISO']
    node_mode        = 3
    t_min            = 0
    t_max            = 1
    dt               = 1/5
    orient_with      = 'XISO'

In [ ]:
Cvec_Brown, *_ = get_materials_Cvec(material="Brown")
Cmat_Brown = v2sm(Cvec_Brown)
Tmat_Brown = tmat_of_cmat(Cmat_Brown)

In [ ]:
Tmat_nodes = []

if node_mode == 1:
    for sigma in node_sequence:
        print(f'adding node {sigma} ....')
        if sigma == "TRIV":
            Tmat_nodes.append(Tmat_Brown)    
        else:
            Tmat_nodes.append(closest(Tmat_Brown, sigma))

elif node_mode == 2:
    for i, sigma in enumerate(node_sequence):
        print(f'adding node {sigma} ....')
        if i==0:
            Tmat_nodes.append(Tmat_Brown)
        else:
            Tmat_nodes.append(closest(Tmat_nodes[i-1], sigma))
            
elif node_mode == 3:
    
    temp = GetTempAndT0S0P0(Tmat=Tmat_Brown, Sigma=orient_with)
    U = UsHat([temp[1]['theta'], 0, temp[1]['phi']])
    
    for i, sigma in enumerate(node_sequence):
        print(f'adding node {sigma} ....')
        if i==0:
            Tmat_nodes.append(Tmat_Brown)
        else:
            Tmat_nodes.append(proj_to_vsig_of_u_new(Tmat_Brown, U, sigma))
    
else:
    print('The chosen node mode is not available')

In [ ]:
num_workers        = os.cpu_count()

npts               = round( (t_max - t_min) / dt + 1 )
n_symmetry_classes = len(symmetry_classes)
n_nodes            = len(node_sequence)
n_c_vecs           = (n_nodes-1) * (npts-1) + 1
data_size          = n_symmetry_classes * n_c_vecs
frac               = np.linspace(t_min,t_max,npts)
betas              = np.zeros((n_symmetry_classes, n_c_vecs)) 

for i in range(n_nodes-1):
    
    t_mat1 = Tmat_nodes[i]
    t_mat2 = Tmat_nodes[i+1]
    
    if pathway==1:    
    
        c_vecs = []
        for j in range(npts):
            t_mat_test = (1-frac[j]) * t_mat1 + (frac[j]) * t_mat2
            c_mat_test = cmat_of_tmat(t_mat_test)
            c_vec_test = sm2v(c_mat_test)
            c_vecs.append(c_vec_test)
    
    elif pathway in [2, 3, 4]:
        
        if i==0:
            c_vecs = [Cvec_Brown]
        
        for j in range(1,npts):
            t_mat_test = (1-frac[j]) * t_mat1 + (frac[j]) * t_mat2
            c_mat_test = cmat_of_tmat(t_mat_test)
            c_vec_test = sm2v(c_mat_test)
            c_vecs.append(c_vec_test)

if data_size < num_workers:

    results = []
    for i, sigma in enumerate(symmetry_classes):
        for j, c_vec in enumerate(c_vecs):
            print(f'running iteration {i+1}, {j+1} ....')
            results.append(distance(c_vec, sigma))

else:
    
    c_vecs   = c_vecs * n_symmetry_classes
    sigmas   = [sigma for sigma in symmetry_classes for _ in range(n_c_vecs)]
    tracker  = range(data_size)
    distance_new = partial(distance, use_parallel_processing=False)       
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(distance_new, c_vecs, sigmas, tracker))

betas_vec, thetas_vec, sigmas_vec, phis_vec = list(zip(*results))

for i in range(n_symmetry_classes):
    index = i * n_c_vecs
    betas[i,:] = betas_vec[index : index + n_c_vecs]

In [ ]:
plt.figure(figsize=(10,10))

if pathway==1:
    
    for i, sym_class in enumerate(symmetry_classes):
        plt.plot(frac, betas[i], '-o', color=colors[sym_class], label=sym_class)
    
    for xc in [t_min, 0, t_max]:
        plt.axvline(x=xc,color='k',linestyle=':')
        
    plt.axhline(y=0, color='k', linestyle=':')
    plt.grid()
    plt.xlabel("t", fontsize=25)
    
elif pathway in [2, 3, 4]:    
    
    x = np.arange(n_c_vecs)
    xticks = np.arange(n_nodes)*(npts-1)
    
    for i, sym_class in enumerate(symmetry_classes):
        plt.plot(x, betas[i], '-o', color=colors[sym_class], label=sym_class)
    
    for xc in np.arange(n_nodes)*(npts-1):
        plt.axvline(x=xc,color='k',linestyle=':')
    
    plt.xticks(xticks, node_sequence)
    plt.xlabel("Node Sequence", fontsize=25)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel(r'$\beta_{\Sigma}$ (°)', fontsize=25)
plt.legend(loc=1, fontsize=13, title=r'${\Sigma}$ (Brown2016)', title_fontsize=15)
if write: plt.savefig(f'Pathway_{pathway}.png')
plt.show()

In [ ]:
# Inspect contour maps
# Make sure that the plotting of the Contour Map is not attempted for the isotropic class

'''
for i in range(n_symmetry_classes):
    for j in range(npts):
        ax = ES_ContourPlots(Tmat, symmetry_classes[i])
        ax.scatter(thetas_vec[i], phis_vec[i], c='r', marker='x')
        plt.show()
'''